import torch
from torch.utils.data import DataLoader

from datautil.waymo_dataset import WaymoDataset, waymo_collate_fn

dataset = WaymoDataset('data')
dataloader = DataLoader(dataset, batch_size=4, collate_fn=lambda x: waymo_collate_fn(x), shuffle=False)

data0 = next(iter(dataloader))

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as FU

from model.utils import *

from collections import OrderedDict

A = 128
B = 64
T = 91
D = 256
F = 6
H = 4
K = 7

Q_ = torch.rand((A,T,D))
R_ = Q_.unsqueeze(0).repeat(F,1,1,1)

onehots_ = torch.tensor(range(F))
onehots_ = FU.one_hot(onehots_, num_classes=F)
onehots_ = onehots_.view(F,1,1,F).repeat(1,A,T,1)
S_ = torch.cat((R_,onehots_),dim=-1)

layer_T = nn.Sequential(nn.Linear(D+F,D), nn.ReLU(), Permute4Batchnorm((0,3,1,2)),
                            nn.BatchNorm2d(D), Permute4Batchnorm((0,2,3,1)))
T_ = layer_T(S_)

In [3]:
print(T_.shape)
print(T_.reshape(-1,T,D).shape)

torch.Size([6, 128, 91, 256])
torch.Size([768, 91, 256])


In [4]:
layer_U = nn.TransformerEncoderLayer(D,H,dim_feedforward=D)

# src_mask = 
U_ = layer_U(T_.reshape(-1,T,D))

In [16]:
layer__ = nn.MultiheadAttention(embed_dim=D,num_heads=H)

padding_mask = torch.rand((A,T))
agent_mask = torch.rand((A,A))

print(T_.shape)

# across time
q = T_.reshape((-1,T,D)).permute(1,0,2)     # [L,N,E] = [T,A*F,D] = [91,768,256]
k, v = q.clone(), q.clone()                 # [S,N,E] = [T,A*F,D]

key_padding_mask = padding_mask.repeat(F,1) # [N,S] = [A*F,T]
attn_mask = None                 

att_output, _ = layer__(q,k,v,key_padding_mask=key_padding_mask,attn_mask=attn_mask)

# across agent
q = T_.reshape((-1,A,D)).permute(1,0,2)     # [L,N,E] = [A,T*F,D] = [546,128,256]
k, v = q.clone(), q.clone()                 # [S,N,E] = [A,T*F,D]

key_padding_mask = padding_mask.permute(1,0).repeat(F,1) # [N,S] = [T*F,A]
attn_mask = agent_mask                      # [L,S] = [A,A]

torch.Size([6, 128, 91, 256])
torch.Size([546, 128])
